In [3]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import math

In [23]:
dir = "E:\\greenland_Campus\\GisDataforChina\\"

## osm

In [4]:
i=10

比较osm与我获得的数据量

In [ ]:
from osgeo import gdal

In [33]:
import warnings

In [40]:
def RoadFromOSM(i):
    # 打开TIFF文件
    dataset = gdal.Open(f'H:\\layers\\river_compressed\\campus_{i}.tif')

    # 获取地理变换信息
    geo_transform = dataset.GetGeoTransform()

    # 计算坐标范围
    east = geo_transform[0]
    west = geo_transform[0] + geo_transform[1] * dataset.RasterXSize
    south = geo_transform[3] + geo_transform[5] * dataset.RasterYSize
    north = geo_transform[3]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", FutureWarning)
        try:
            graph = ox.graph_from_bbox(north, south, east, west)
        except:
            return 0,0

        # 绘制街道网络
        gdf_edges = ox.graph_to_gdfs(graph)[1]
        length_osm = gdf_edges['length'].sum()
        count_osm = len(gdf_edges)

        gdf_edges.to_file(dir+f'\\OSM_road\\OSM_{i}.gpkg',driver='GPKG')
    dataset = None
    return count_osm,length_osm


In [38]:
from concurrent.futures import ThreadPoolExecutor, as_completed,ProcessPoolExecutor

In [54]:
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(RoadFromOSM, i) for i in range(12,71)]
    for future in enumerate(futures):
        counts[i], lengths[i] = future.result()

In [64]:
counts = {}
lengths = {}

In [65]:
import os 
for i in range(1263):
    file = [dir+'OSM_road\\OSM_{}.gpkg'.format(i) ]
    if os.path.exists(file[0])==False:
        counts[i]=0
        lengths[i]=0
    else:
        osm_gdf = gpd.read_file(file[0])
        counts[i] = len(osm_gdf)
        lengths[i] = osm_gdf['length'].sum()



In [68]:
aoi_context = gpd.read_file(dir+'AOI_gpkg\\aoi_contextIndex.gpkg')
aoi_context.head()

,面积Albers,aspect_ratio,orient,areaUTM,campusName,degree,length,shapeIndex,geometry
0,3.892316e+05,1.032993,0.191192,3.891525e+05,海南热带海洋学院,5.511174,2586.522202,1.036564,"POLYGON ((109.53224 18.31567, 109.53225 18.315..."
1,1.088431e+06,1.542657,0.028879,1.088187e+06,三亚学院(北校区),90.827440,5339.996905,1.279762,"POLYGON ((109.55245 18.34337, 109.55246 18.343..."
2,4.899977e+05,1.145554,0.072357,4.896646e+05,海南大学(观澜湖校区),87.925298,2900.564722,1.036271,"POLYGON ((110.33655 19.92597, 110.33656 19.925..."
3,8.635738e+05,1.532838,0.696911,8.629453e+05,海口经济学院,157.910137,5054.089198,1.360164,"POLYGON ((110.48029 19.97086, 110.48022 19.970..."
4,6.810645e+05,1.168341,0.610504,6.805635e+05,琼台师范学院(桂林洋校区),161.187024,3302.370708,1.000764,"POLYGON ((110.51748 19.98019, 110.51749 19.980..."


In [69]:
aoi_context['osm_count'] = counts
aoi_context['osm_length'] = lengths

### 统计范围内的道路

In [91]:
lengths_weighted={}
counts_weighted={}

In [84]:
import numpy as np

In [92]:
def SumRoadFromTile(i):
    road_gdf = gpd.read_file(dir+'road_gpkg\\1_centerLine_road_{}.gpkg'.format(i), layer='road')
    length=road_gdf.length.sum()
    road_gdf['weight']= np.where(road_gdf['DN'] > 2, 2, 1)
    length_weighted = (road_gdf.length*road_gdf['weight']).sum()
    count=len(road_gdf)
    count_weighted=road_gdf['weight'].sum()

    return length,length_weighted,count,count_weighted

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(SumRoadFromTile, i) for i in range(1263)]
    for i,future in enumerate(futures):
        lengths[i],lengths_weighted[i],counts[i],counts_weighted[i] = future.result()


In [93]:
aoi_context['road_count'] = counts
aoi_context['road_weighted_count'] = counts_weighted
aoi_context['road_length'] = lengths
aoi_context['road_weighted_length'] = lengths_weighted

In [99]:
aoi_context.to_file(dir+'AOI_gpkg\\aoi_contextIndex.gpkg',driver='GPKG')

### 统计范围内的绿地

In [100]:
t = 'green'
areas = {}
counts = {}
def SumBuildingFromTile(i):
    gdf = gpd.read_file(dir+f'{t}_gpkg\\1_inside_{t}_{i}.gpkg')
    area=gdf.area.sum()
    count=len(gdf)
    return area,count

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(SumBuildingFromTile, i) for i in range(1263)]
    for i,future in enumerate(futures):
        areas[i],counts[i]= future.result()

In [103]:
areas[0]

96694.39836398653

In [102]:
aoi_context=gpd.read_file(dir+'AOI_gpkg\\aoi_contextIndex.gpkg')
aoi_context

,面积Albers,aspect_ratio,orient,areaUTM,campusName,degree,length,shapeIndex,osm_count,osm_length,road_count,road_length,road_weighted_length,road_weighted_count,building_area,building_count,geometry
0,3.892316e+05,1.032993,0.191192,3.891525e+05,海南热带海洋学院,5.511174,2586.522202,1.036564,165,35799.634,624,65457.580343,76217.185053,742,111352.391399,37,"POLYGON ((109.53224 18.31567, 109.53225 18.315..."
1,1.088431e+06,1.542657,0.028879,1.088187e+06,三亚学院(北校区),90.827440,5339.996905,1.279762,260,40730.232,536,62150.032875,65703.337493,572,108045.906314,124,"POLYGON ((109.55245 18.34337, 109.55246 18.343..."
2,4.899977e+05,1.145554,0.072357,4.896646e+05,海南大学(观澜湖校区),87.925298,2900.564722,1.036271,228,41495.488,665,60016.703020,65136.705248,718,3569.027558,12,"POLYGON ((110.33655 19.92597, 110.33656 19.925..."
3,8.635738e+05,1.532838,0.696911,8.629453e+05,海口经济学院,157.910137,5054.089198,1.360164,129,27793.658,423,53501.827663,56284.564563,435,156815.887941,72,"POLYGON ((110.48029 19.97086, 110.48022 19.970..."
4,6.810645e+05,1.168341,0.610504,6.805635e+05,琼台师范学院(桂林洋校区),161.187024,3302.370708,1.000764,276,46464.536,820,70369.492461,73750.891623,839,64554.718650,51,"POLYGON ((110.51748 19.98019, 110.51749 19.980..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258,4.481313e+05,1.555779,0.814267,4.477935e+05,齐齐哈尔理工职业学院,152.742502,2730.198221,1.019988,118,32171.828,534,43021.704354,46791.030616,561,39068.039941,49,"POLYGON ((123.59605 47.21736, 123.59606 47.217..."
1259,8.996020e+05,1.630024,0.399819,9.001826e+05,华南农业大学（东区）,168.216578,4869.680996,1.283143,1077,126434.909,1488,101699.994150,119715.414695,1791,87473.329896,237,"POLYGON ((113.35701 23.15479, 113.35782 23.156..."
1260,1.453719e+06,1.478692,0.079176,1.455402e+06,浙江大学(紫金港校区西区),92.270599,4872.589498,1.009737,1627,133135.351,1006,81934.360173,100658.693936,1256,192464.198871,149,"POLYGON ((120.07706 30.29731, 120.07629 30.297..."
1261,9.848053e+05,1.751273,0.633265,9.845656e+05,上海交通大学(闵行校区),109.645698,4345.746775,1.094919,2020,149223.904,1000,81832.330743,95054.748667,1121,115368.811737,169,"POLYGON ((121.43059 31.02074, 121.42909 31.020..."


In [104]:
aoi_context['green_area'] = areas
aoi_context['green_count'] = counts
aoi_context

,面积Albers,aspect_ratio,orient,areaUTM,campusName,degree,length,shapeIndex,osm_count,osm_length,road_count,road_length,road_weighted_length,road_weighted_count,building_area,building_count,geometry,green_area,green_count
0,3.892316e+05,1.032993,0.191192,3.891525e+05,海南热带海洋学院,5.511174,2586.522202,1.036564,165,35799.634,624,65457.580343,76217.185053,742,111352.391399,37,"POLYGON ((109.53224 18.31567, 109.53225 18.315...",96694.398364,24
1,1.088431e+06,1.542657,0.028879,1.088187e+06,三亚学院(北校区),90.827440,5339.996905,1.279762,260,40730.232,536,62150.032875,65703.337493,572,108045.906314,124,"POLYGON ((109.55245 18.34337, 109.55246 18.343...",85910.822703,18
2,4.899977e+05,1.145554,0.072357,4.896646e+05,海南大学(观澜湖校区),87.925298,2900.564722,1.036271,228,41495.488,665,60016.703020,65136.705248,718,3569.027558,12,"POLYGON ((110.33655 19.92597, 110.33656 19.925...",23986.194844,13
3,8.635738e+05,1.532838,0.696911,8.629453e+05,海口经济学院,157.910137,5054.089198,1.360164,129,27793.658,423,53501.827663,56284.564563,435,156815.887941,72,"POLYGON ((110.48029 19.97086, 110.48022 19.970...",435713.398089,99
4,6.810645e+05,1.168341,0.610504,6.805635e+05,琼台师范学院(桂林洋校区),161.187024,3302.370708,1.000764,276,46464.536,820,70369.492461,73750.891623,839,64554.718650,51,"POLYGON ((110.51748 19.98019, 110.51749 19.980...",498503.358337,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258,4.481313e+05,1.555779,0.814267,4.477935e+05,齐齐哈尔理工职业学院,152.742502,2730.198221,1.019988,118,32171.828,534,43021.704354,46791.030616,561,39068.039941,49,"POLYGON ((123.59605 47.21736, 123.59606 47.217...",168691.887430,28
1259,8.996020e+05,1.630024,0.399819,9.001826e+05,华南农业大学（东区）,168.216578,4869.680996,1.283143,1077,126434.909,1488,101699.994150,119715.414695,1791,87473.329896,237,"POLYGON ((113.35701 23.15479, 113.35782 23.156...",143019.030584,51
1260,1.453719e+06,1.478692,0.079176,1.455402e+06,浙江大学(紫金港校区西区),92.270599,4872.589498,1.009737,1627,133135.351,1006,81934.360173,100658.693936,1256,192464.198871,149,"POLYGON ((120.07706 30.29731, 120.07629 30.297...",309148.552651,383
1261,9.848053e+05,1.751273,0.633265,9.845656e+05,上海交通大学(闵行校区),109.645698,4345.746775,1.094919,2020,149223.904,1000,81832.330743,95054.748667,1121,115368.811737,169,"POLYGON ((121.43059 31.02074, 121.42909 31.020...",249954.869410,38
